# Ingest Treehouse

Download the [UCSC Treehouse](https://treehousegenomics.soe.ucsc.edu/public-data/) compendium RNA-Seq expression and clinical labels from [Xena](https://xenabrowser.net), wrangle, prune and store in a single h5 file for use in the training notebook.

In [1]:
import os
import numpy as np
import pandas as pd

# Switch to a scratch data directory so all paths are local
os.makedirs(os.path.expanduser("~/data/treehouse"), exist_ok=True)
os.chdir(os.path.expanduser("~/data/treehouse"))

## Ingest Samples

In [2]:
# Download raw files from xena
!wget -q -N https://xena.treehouse.gi.ucsc.edu/download/TreehousePEDv8_unique_hugo_log2_tpm_plus_1.2018-07-25.tsv

In [3]:
%%time
# Convert to float32, Transpose to ML style rows = samples and hdf for significantly faster reading
if not os.path.exists(os.path.expanduser("treehouse.T.fp32.h5")):
    pd.read_csv(os.path.expanduser("TreehousePEDv8_unique_hugo_log2_tpm_plus_1.2018-07-25.tsv"), 
                sep="\t", index_col=0, engine='c') \
        .astype(np.float32).T \
        .to_hdf(os.path.expanduser("treehouse.T.fp32.h5"), "expression", mode="w", format="fixed")

CPU times: user 43 µs, sys: 55 µs, total: 98 µs
Wall time: 107 µs


In [4]:
# Convert back to TPM
all_samples = pd.read_hdf("treehouse.T.fp32.h5").apply(np.exp2).subtract(1.0).clip(lower=0.0)
print("Ingested {} samples with {} features".format(all_samples.shape[0], all_samples.shape[1]))
all_samples.head()

Ingested 11427 samples with 58581 features


Gene,5S_rRNA,5_8S_rRNA,7SK,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,...,snoU2-30,snoU2_19,snoU83B,snoZ196,snoZ278,snoZ40,snoZ6,snosnR66,uc_338,yR211F11.2
THR15_0330_S01,0.0,0.0,0.00,5.140000,1.220000,0.00,147.210022,0.25,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.500000,0.00
THR29_0776_S01,0.0,0.0,0.06,31.359997,4.030000,0.00,39.079998,0.93,0.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.470000,0.00
THR14_1221_S01,0.0,0.0,0.78,2.680000,1.730000,0.01,28.330002,0.25,0.40,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.510000,0.15
THR11_0247_S01,0.0,0.0,1.94,8.280001,8.580001,0.14,465.710022,1.05,0.23,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.400000,0.00
THR08_0162_S01,0.0,0.0,0.00,17.389999,21.900002,0.00,0.730000,0.12,0.14,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.880005,0.00


In [5]:
# Check that TPM (transcripts per million) per sample sum to 1M
all_samples.iloc[::all_samples.shape[0]//5].sum(axis=1)

THR15_0330_S01     1.000000e+06
TCGA-VF-A8AE-01    1.000003e+06
TCGA-BR-8080-01    1.000001e+06
TCGA-IB-AAUP-01    1.000001e+06
TCGA-E2-A1IE-01    1.000004e+06
TCGA-BR-8588-01    1.000006e+06
dtype: float64

## Prune Features
Reduce the number of features/genes from ~20k to ~5k so that the samples to features ratio is more in line with typical machine learning data sets. If there were significantly mroe training samples this could be purely computational but for now we'll use COSMIC+KEGG genes to 'feature engineer' leveraging current biological knowledge. This also reduces the size of the background file required when computing the explantion for a classification so that it can be run in resonable time in a Colab notebook.

In [6]:
# Load KEGG pathway genes (http://software.broadinstitute.org/gsea/msigdb/)
with open("c2.cp.kegg.v6.2.symbols.gmt") as f:
    kegg_genes = list(set().union(*[line.strip().split("\t")[2:] for line in f.readlines()]))

# Load COSMIC Cancer Genes (https://cancer.sanger.ac.uk/census)
cosmic_genes = pd.read_csv("cosmic-26-11-2018.tsv", sep="\t")["Gene Symbol"].values

# Prune to only genes in KEGG and COSMIC
subset_of_genes = list(set(kegg_genes).union(set(cosmic_genes)))

pruned_samples = all_samples.drop(labels=(set(all_samples.columns) - set(subset_of_genes)), axis=1)

print("Pruning from {} down to {} features/genes".format(all_samples.shape[1], pruned_samples.shape[1]))

Pruning from 58581 down to 5545 features/genes


## Ingest Labels

In [7]:
!wget -q -N https://xena.treehouse.gi.ucsc.edu/download/TreehousePEDv8_clinical_metadata.2018-07-25.tsv

In [8]:
all_labels = pd.read_csv("TreehousePEDv8_clinical_metadata.2018-07-25.tsv", 
                         header=0, sep="\t", index_col=0).sort_index(axis="index")

print(all_labels.dtypes)

all_labels.iloc[::all_labels.shape[0]//5]

disease       object
age_at_dx    float64
pedaya        object
gender        object
dtype: object


,disease,age_at_dx,pedaya,gender
th_sampleid,,,,
TARGET-10-PAKSWW-03,acute lymphoblastic leukemia,15.11,"Yes, age < 30 years",male
TCGA-66-2757-01,lung squamous cell carcinoma,65.00,No,female
TCGA-C5-A3HD-01,cervical & endocervical cancer,51.00,No,female
TCGA-EK-A2RC-01,cervical & endocervical cancer,33.00,No,female
TCGA-P8-A5KC-01,pheochromocytoma & paraganglioma,48.00,No,male
THR33_1146_S01,medulloblastoma,8.00,"Yes, age < 30 years",male


## Prune

Drop samples with missing values for fields we want to train on, or that have too few of the class and transform field values for training etc...

In [9]:
# Include only labels for samples that we have
pruned_labels = all_labels.loc[all_labels.index.intersection(pruned_samples.index)]
print("Starting with {} labeled sample pairs".format(pruned_labels.shape[0]))

# Drop any missing the label we plan to classify
pruned_labels = pruned_labels.dropna(subset=["disease"])
print(pruned_labels.shape[0], "with disease")

# Drop the other columns as they may have NaN or mixed types which will have issues later exporting
pruned_labels = pruned_labels[["disease"]]

# Drop diseases with less then 50 examples
counts = pruned_labels.disease.value_counts()
pruned_labels = pruned_labels[pruned_labels.disease.isin(counts[counts >= 100].index)]
print("Dropping {} labels where the disease has < 100 samples".format(counts[counts < 100].index.shape[0]))
print(pruned_labels.shape[0], "with >= 100 samples of the disease")

print("Number of unique labels (diseases):", len(pruned_labels.disease.unique()))

print("{} labels after pruning".format(pruned_labels.shape[0]))
pruned_labels.iloc[::pruned_labels.shape[0]//5]

Starting with 11427 labeled sample pairs
11426 with disease
Dropping 66 labels where the disease has < 100 samples
10208 with >= 100 samples of the disease
Number of unique labels (diseases): 29
10208 labels after pruning


,disease
TARGET-10-PAKSWW-03,acute lymphoblastic leukemia
TCGA-56-A49D-01,lung squamous cell carcinoma
TCGA-BJ-A18Z-01,thyroid carcinoma
TCGA-E2-A14P-01,breast invasive carcinoma
TCGA-KK-A8IL-01,prostate adenocarcinoma
THR33_1145_S01,medulloblastoma


## Export

Export the pruned dataset with labels as an h5 file.

In [10]:
%%time
# Include only ids that we have both sample and label for
sample_ids = pruned_samples.index.intersection(pruned_labels.index)
print("Exporting {} samples".format(len(sample_ids)))

# NOTE: Setting complevel to 9 reduces the size of the resulting h5 file from 3G down to 2.1G
# but increases the read time from 2.79s to 20.8s and the write time from 19.9s to 25m
pruned_samples.loc[sample_ids].sort_index(axis="index").sort_index(axis="columns").to_hdf(
    os.path.expanduser("treehouse-pruned.h5"), key="samples", mode="w", format="fixed", complevel=0)
pruned_labels.loc[sample_ids].sort_index(axis="index").sort_index(axis="columns").to_hdf(
    os.path.expanduser("treehouse-pruned.h5"), key="labels", mode="a", format="fixed", complevel=0)

Exporting 10208 samples
CPU times: user 4.12 s, sys: 3.17 s, total: 7.29 s
Wall time: 7.78 s
